# How do I get data into my notebook?

In [ ]:
# Importing CSVs and Excel using Pandas
import pandas as pd

In [ ]:
# First, we'll import a CSV.
df_csv = pd.read_csv('Movie_Actors.csv') 
df_csv.head()

In [ ]:
# Then we'll import an Excel file
df_excel = pd.read_excel("Fortune1000.xlsx")
df_excel.head()